In [24]:
import numpy as np
import matplotlib.pyplot as plt

In [40]:
#Constants for the Code

V2 = 0
V3 = 256000 #Newtons
E_al = 10.5*10**3 * 6.89476 #Mpa #Youngs Modulus of aluminium, the material of stringer
#0.020 − 0.062𝑖𝑛 thickness range for the “L” direction
R = 1.95 #m #Distance from origion to LE
A_s = 150 * (1/1000)**2 #m^2 #Stringer Area
t = 2.95/1000 #m #Thickness
c = 8*R #Chord

x2i = np.array([
    7*R,
    R,
    0,
    -R,
    -2*R/3,
    R
])

x3i = np.array([
    0,
    7/8*R,
    R,
    0,
    -R/3,
    -6*R/23
])
H22c = 0
H33c = 0
H23c = 0
#H22c

for i in range(np.size(x2i)):
    H22c += E_al*A_s*(x3i[i])**2
    H23c += E_al*A_s*(x3i[i])*(x2i[i])
    H33c += E_al*A_s*(x2i[i])**2
#This method is faster and only yields errors in the float, ie e-15

print("H22c: ", H22c)
print("H23c: ", H23c)
print("H33c: ", H33c)

del_fi = np.array([]) #shear flow due to the presence of stringers, i

for i in range(np.size(x2i)):
    shearflowinx2 = x2i[i]*(H22c*V2 - H23c*V3)/(H22c*H33c - (H23c)**2)
    shearflowinx3 = x3i[i]*(H23c*V2 - H33c*V3)/(H22c*H33c - (H23c)**2)

    calc = -E_al*A_s*( shearflowinx2 - shearflowinx3)
    del_fi = np.append(del_fi, calc)
print(del_fi)

#We make a cut along 12
#therefore, our shearflow in the skin calcs start at 2
f_ij = np.array([])
cut = 2

#for i in 
#f_ij.insert(cut-1,0)
#for i in range(np.size(x2i)):
#    f_ij.insert(cut,0)
my_list = x2i
#print(x2i)
#for idx, val in enumerate(my_list, start=2):
#    print(f"At index {idx}, the item is {val}")

def update_array(x, start_index, update_func):
    n = len(x)
    i = start_index
    
    while True:
        # Update the element at index i
        x[i] = update_func(x[i])  

        # Move to the next index in a circular fashion
        i = (i + 1) % n  

        # Stop when we reach the index before start_index
        if i == (start_index - 1) % n:
            break  

    return x

# Example usage:
arr = [0, 2, 3, 4, 5, 6]
start_index = 0

# Define an update function (e.g., increment each element)
update_func = lambda x: x + 10

updated_arr = update_array(arr, start_index, update_func)
print(updated_arr)

    


H22c:  80.3047866929536
H23c:  34.53491370998641
H33c:  2165.5510367399993
[  7587.68972603 -58390.38343374 -67970.6732673   -1083.95567515
  21934.25397233  18815.43565792]
[10, 12, 13, 14, 15, 6]
